In [9]:
from bs4 import BeautifulSoup
import requests

In [10]:
response = requests.get("https://www.tnwb.uscourts.gov/Search/search.aspx?zoom_query=CAR")
doc = BeautifulSoup(response.text, 'html.parser')

In [11]:
items = doc.find_all('div', class_=['result_block', 'result_altblock'])

In [16]:
results = []

In [17]:
for item in items:
    case_data = {
        'Name': item.find('a').text.strip(),
        'URL': "https://www.tnwb.uscourts.gov" + item.find('a')['href'],
        'Category': item.find('span', class_='category').text.strip(),
        'Details': item.find('div', class_='infoline').text.strip()
    }
    results.append(case_data)

In [18]:
df = pd.DataFrame(results)

In [19]:
print(df.head())

                                           Name  \
0            JDL: 04-24318 Jacquelline D. Black   
1                WHB: 95-26401 Mary Lucy Cooper   
2                GHB: 97-12368 Billy G. Woffard   
3               JDL: 97-30580 Mary Chrlis Hurst   
4  MRH: 20-20967 Jacob Braxton Herring 20-00094   

                                                 URL            Category  \
0  https://www.tnwb.uscourts.govhttps://www.tnwb....  [Judges' Opinions]   
1  https://www.tnwb.uscourts.govhttps://www.tnwb....  [Judges' Opinions]   
2  https://www.tnwb.uscourts.govhttps://www.tnwb....  [Judges' Opinions]   
3  https://www.tnwb.uscourts.govhttps://www.tnwb....  [Judges' Opinions]   
4  https://www.tnwb.uscourts.govhttps://www.tnwb....  [Judges' Opinions]   

                                             Details  
0  Terms matched:  1  -  102k  -  URL: https://ww...  
1  Terms matched:  1  -  27k  -  URL: https://www...  
2  Terms matched:  1  -  71k  -  URL: https://www...  
3  Terms matched

In [20]:
df.to_csv('car_case_results.csv', index=False)

In [22]:
for case in results:
    if "URL:" in case['Details']:
        pdf_url = case['Details'].split("URL:")[1].strip()
        try:
            pdf_response = requests.get(pdf_url)
            filename = f"{case['Name'][:30]}.pdf".replace('/', '_')  # Safe filename
            with open(filename, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Downloaded {filename}")
        except:
            print(f"Failed to download PDF for {case['Name']}")

Downloaded JDL: 04-24318 Jacquelline D. B.pdf
Downloaded WHB: 95-26401 Mary Lucy Cooper.pdf
Downloaded GHB: 97-12368 Billy G. Woffard.pdf
Downloaded JDL: 97-30580 Mary Chrlis Hurs.pdf
Downloaded MRH: 20-20967 Jacob Braxton He.pdf
Downloaded GHB: 95-11365 Melissa L. Bryan.pdf
Downloaded JDL: 09-20339 Diane M. Miller.pdf
Downloaded GHB: 00-12340 Wanda K. Autry.pdf
Downloaded GHB: 02-31651 Neil Bond Stewar.pdf
Downloaded GHB: 96-12039 Randy and Janice.pdf
